In [1]:
!ls

OD_ingest.png			    ilsvrc_2012_mean.npy
Object detection.ipynb		    imagenet.bet.pickle
Untitled.ipynb			    imagenet_mean.binaryproto
alexnet_layers.png		    jupyter_executing.png
bvlc_reference_caffenet.caffemodel  jupyter_idle.png
caffe_ilsvrc12.tar.gz		    kespry_example.png
candidate_generation_example.png    overfeat.png
countdown_v5.0			    patch_based.png
data				    right_whale_example.png
data_336x224			    synset_words.txt
deploy.prototxt			    synsets.txt
deploy.prototxt.1		    test.txt
deploy.prototxt.2		    train.txt
det_synset_words.txt		    val.txt
detectnet.prototxt		    whale_face_examples.png
detectnet_data.png		    whale_faces_digits_model.png
detectnet_success.png		    whale_faces_import.png
detectnet_training.png		    yolo.png


In [2]:
!rm -rf ilsvrc_2012_mean.npy.1

In [6]:
!wget http://dl.caffe.berkeleyvision.org/bvlc_reference_caffenet.caffemodel

--2017-01-24 15:56:18--  http://dl.caffe.berkeleyvision.org/bvlc_reference_caffenet.caffemodel
Resolving dl.caffe.berkeleyvision.org (dl.caffe.berkeleyvision.org)... 169.229.222.251
Connecting to dl.caffe.berkeleyvision.org (dl.caffe.berkeleyvision.org)|169.229.222.251|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 243862418 (233M) [application/octet-stream]
Saving to: 'bvlc_reference_caffenet.caffemodel'

100%[======================================>] 243,862,418 7.27MB/s   in 1m 53s 

2017-01-24 15:58:13 (2.05 MB/s) - 'bvlc_reference_caffenet.caffemodel' saved [243862418/243862418]



In [3]:
import sys
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (10, 10)        # large images
plt.rcParams['image.interpolation'] = 'nearest'  # don't interpolate: show square pixels
plt.rcParams['image.cmap'] = 'gray'  # use grayscale output rather than a  color heatmap

import caffe

param_root = './'
caffe_root ='/home/ubuntu/caffe/'
import os
if os.path.isfile(param_root + 'bvlc_reference_caffenet.caffemodel'):
    print 'CaffeNet found.'
else:
    print 'Need to download pre-trained CaffeNet model...'
    
   

CaffeNet found.


In [4]:
caffe.set_mode_gpu() 

In [14]:
!head -n 20 deploy.prototxt.1

name: "CaffeNet"
layer {
  name: "data"
  type: "Input"
  top: "data"
  input_param { shape: { dim: 10 dim: 3 dim: 227 dim: 227 } }
}
layer {
  name: "conv1"
  type: "Convolution"
  bottom: "data"
  top: "conv1"
  convolution_param {
    num_output: 96
    kernel_size: 11
    stride: 4
  }
}
layer {
  name: "relu1"


In [15]:
!wget https://raw.githubusercontent.com/BVLC/caffe/master/models/bvlc_reference_caffenet/deploy.prototxt

--2017-01-24 16:06:50--  https://raw.githubusercontent.com/BVLC/caffe/master/models/bvlc_reference_caffenet/deploy.prototxt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.12.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.12.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2878 (2.8K) [text/plain]
Saving to: 'deploy.prototxt.2'

100%[======================================>] 2,878       --.-K/s   in 0s      

2017-01-24 16:06:50 (85.5 MB/s) - 'deploy.prototxt.2' saved [2878/2878]



In [5]:
model_def = param_root + 'deploy.prototxt.1'
model_weights = param_root + 'bvlc_reference_caffenet.caffemodel'
net = caffe.Net(model_def,      # defines the structure of the model
                model_weights,  # contains the trained weights
                caffe.TEST)     # use test mode (e.g., don't perform dropout)


In [27]:
!wget https://raw.githubusercontent.com/BVLC/caffe/master/python/caffe/imagenet/ilsvrc_2012_mean.npy

--2017-01-24 16:12:31--  https://raw.githubusercontent.com/BVLC/caffe/master/python/caffe/imagenet/ilsvrc_2012_mean.npy
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.12.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.12.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1572944 (1.5M) [application/octet-stream]
Saving to: 'ilsvrc_2012_mean.npy'

100%[======================================>] 1,572,944   --.-K/s   in 0.04s   

2017-01-24 16:12:31 (41.9 MB/s) - 'ilsvrc_2012_mean.npy' saved [1572944/1572944]



In [6]:
# load the mean ImageNet image (as distributed with Caffe) for subtraction
mu = np.load(param_root + 'ilsvrc_2012_mean.npy')
mu = mu.mean(1).mean(1)  # average over pixels to obtain the mean (BGR) pixel values
print 'mean-subtracted values:', zip('BGR', mu)

# create transformer for the input called 'data'
transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})

transformer.set_transpose('data', (2,0,1))  # move image channels to outermost dimension
transformer.set_mean('data', mu)            # subtract the dataset-mean value in each channel
transformer.set_raw_scale('data', 255)      # rescale from [0, 1] to [0, 255]
transformer.set_channel_swap('data', (2,1,0))  # swap channels from RGB to BGR

mean-subtracted values: [('B', 104.0069879317889), ('G', 116.66876761696767), ('R', 122.6789143406786)]


In [7]:
net.blobs['data'].reshape(50,        # batch size
                          3,         # 3-channel (BGR) images
                          227, 227)  # image size is 227x227

In [8]:
image = caffe.io.load_image(caffe_root + 'examples/images/cat.jpg')
transformed_image = transformer.preprocess('data', image)
#plt.imshow(image)

In [9]:
!ls /home/ubuntu/caffe/examples/images/

cat.jpg  cat_gray.jpg  fish-bike.jpg


In [10]:
# for each layer, show the output shape  -- batch size -- channel x memeory size
for layer_name, blob in net.blobs.iteritems():
    print layer_name + '\t' + str(blob.data.shape)

data	(50, 3, 227, 227)
conv1	(10, 96, 55, 55)
pool1	(10, 96, 27, 27)
norm1	(10, 96, 27, 27)
conv2	(10, 256, 27, 27)
pool2	(10, 256, 13, 13)
norm2	(10, 256, 13, 13)
conv3	(10, 384, 13, 13)
conv4	(10, 384, 13, 13)
conv5	(10, 256, 13, 13)
pool5	(10, 256, 6, 6)
fc6	(10, 4096)
fc7	(10, 4096)
fc8	(10, 1000)
prob	(10, 1000)


In [11]:
for layer_name, param in net.params.iteritems():
    print layer_name + '\t' + str(param[0].data.shape), str(param[1].data.shape)

conv1	(96, 3, 11, 11) (96,)
conv2	(256, 48, 5, 5) (256,)
conv3	(384, 256, 3, 3) (384,)
conv4	(384, 192, 3, 3) (384,)
conv5	(256, 192, 3, 3) (256,)
fc6	(4096, 9216) (4096,)
fc7	(4096, 4096) (4096,)
fc8	(1000, 4096) (1000,)


In [12]:
plt.imshow(image)

RuntimeError: Invalid DISPLAY variable

In [13]:

# copy the image data into the memory allocated for the net
net.blobs['data'].data[...] = transformed_image

### perform classification
output = net.forward()

output_prob = output['prob'][0]  # the output probability vector for the first image in the batch

print 'predicted class is:', output_prob.argmax()

predicted class is: 281


In [15]:

# load ImageNet labels
labels_file = param_root + 'synset_words.txt'
if not os.path.exists(labels_file):
    !/home/ubuntu/caffe/data/ilsvrc12/get_ilsvrc_aux.sh
    
labels = np.loadtxt(labels_file, str, delimiter='\t')

print 'output label:', labels[output_prob.argmax()]

output label: n02123045 tabby, tabby cat


In [16]:

# sort top five predictions from softmax output
top_inds = output_prob.argsort()[::-1][:5]  # reverse sort and take five largest items

print 'probabilities and labels:'
zip(output_prob[top_inds], labels[top_inds])

probabilities and labels:


[(0.31243563, 'n02123045 tabby, tabby cat'),
 (0.23797134, 'n02123159 tiger cat'),
 (0.12387209, 'n02124075 Egyptian cat'),
 (0.10075706, 'n02119022 red fox, Vulpes vulpes'),
 (0.07095705, 'n02127052 lynx, catamount')]

In [18]:
def vis_square(data):
    """Take an array of shape (n, height, width) or (n, height, width, 3)
       and visualize each (height, width) thing in a grid of size approx. sqrt(n) by sqrt(n)"""
    
    # normalize data for display
    data = (data - data.min()) / (data.max() - data.min())
    
    # force the number of filters to be square
    n = int(np.ceil(np.sqrt(data.shape[0])))
    padding = (((0, n ** 2 - data.shape[0]),
               (0, 1), (0, 1))                 # add some space between filters
               + ((0, 0),) * (data.ndim - 3))  # don't pad the last dimension (if there is one)
    data = np.pad(data, padding, mode='constant', constant_values=1)  # pad with ones (white)
    
    # tile the filters into an image
    data = data.reshape((n, n) + data.shape[1:]).transpose((0, 2, 1, 3) + tuple(range(4, data.ndim + 1)))
    data = data.reshape((n * data.shape[1], n * data.shape[3]) + data.shape[4:])
    
    plt.imshow(data); plt.axis('off')

In [19]:
# the parameters are a list of [weights, biases]
filters = net.params['conv1'][0].data
vis_square(filters.transpose(0, 2, 3, 1))

RuntimeError: Invalid DISPLAY variable